In [ ]:
using Pkg
Pkg.instantiate()
using DrugResponseModel

# import data from the path
conc_l, pop, g2, g1, g2_0, g1_0 = setup_data("lapatinib");

In [ ]:
params_ode = zeros(4, 8)

for i in 1:8
    fitness, params_ode[:, i] = ODEoptimizer(0.5*ones(4), i, g1, g2, g1_0, g2_0, 2, 2)
    println(fitness)
end

In [ ]:
ODEplot_all(params_ode, g1, g2, g1_0, g2_0, pop, 2, 2)